In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()

print("Workspace name: " + ws.name,
        "Azure region: " + ws.location,
        "Subscription id: " + ws.subscription_id,
        "Resource Group: " + ws.resource_group, sep="\n"
)

Workspace name: labuser25-ml-001
Azure region: swedencentral
Subscription id: f5bc93f2-df0a-4b1a-ab9e-2b0203fc7d26
Resource Group: rg25


In [2]:
from azureml.core import Experiment

experiment = Experiment(workspace=ws, name="diabetes-experiment")

In [3]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import pandas as pd

#Sklearn에서 데이터 로드
diabetes = load_diabetes()

#데이터프레임으로 변환
x_df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y_df = pd.Series(diabetes.target, name="Y")

# train/test 데이터의 분할
x_train, x_test, y_train, y_test = train_test_split(
                                                        x_df, 
                                                        y_df, 
                                                        test_size = 0.2,
                                                        random_state=66)

print(x_train)

          age       sex       bmi        bp        s1        s2        s3  \
440 -0.045472 -0.044642  0.039062  0.001215  0.016318  0.015283 -0.028674   
389 -0.005515  0.050680  0.001339 -0.084856 -0.011201 -0.016658  0.048640   
5   -0.092695 -0.044642 -0.040696 -0.019442 -0.068991 -0.079288  0.041277   
289 -0.074533  0.050680  0.055229 -0.040099  0.053469  0.053174 -0.043401   
101  0.016281  0.050680 -0.045007  0.063187  0.010815 -0.000374  0.063367   
..        ...       ...       ...       ...       ...       ...       ...   
122  0.048974  0.050680  0.081097  0.021872  0.043837  0.064134 -0.054446   
51   0.059871  0.050680  0.016428  0.028758 -0.041472 -0.029184 -0.028674   
119  0.016281 -0.044642 -0.047163 -0.002228 -0.019456 -0.042963  0.033914   
316  0.016281  0.050680  0.014272  0.001215  0.001183 -0.021355 -0.032356   
20  -0.049105 -0.044642 -0.056863 -0.043542 -0.045599 -0.043276  0.000779   

           s4        s5        s6  
440  0.026560  0.044529 -0.025930  
389

In [4]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import joblib
import math

alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

for alpha in alphas:
    #데이터 실험의 시작
    run = experiment.start_logging()
    run.log("alpha value", alpha)

    model = Ridge(alpha = alpha)
    model.fit(X= x_train, y= y_train)

    y_pred = model.predict(x_test)
    rmse = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))
    run.log("rmse", rmse)

    model_name = "model_alpha_"+str(alpha) +".pkl"
    filename = "outputs/" + model_name
    
    joblib.dump(value = model, filename = filename)
    run.upload_file(name= model_name, path_or_stream = filename)

    run.complete()
    print(f"{alpha} experiment completed")


0.1 experiment completed
0.2 experiment completed
0.3 experiment completed
0.4 experiment completed
0.5 experiment completed
0.6 experiment completed
0.7 experiment completed
0.8 experiment completed
0.9 experiment completed
1 experiment completed


In [5]:
!pip install joblib

In [6]:
experiment

Name,Workspace,Report Page,Docs Page
diabetes-experiment,labuser25-ml-001,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
# Best model
minimum_rmse_runid = None
minimum_rmse = None
for run in experiment.get_runs():
    run_metrics = run.get_metrics()
    run_details = run.get_details()
    # each logged metric becomes a key in this returned dict
    run_rmse = run_metrics["rmse"]
    run_id = run_details["runId"]
    print(run_id, run_rmse)
    if minimum_rmse is None:
        minimum_rmse = run_rmse
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse:
            minimum_rmse = run_rmse
            minimum_rmse_runid = run_id
print("-"*100)
print("Best run_id: " + minimum_rmse_runid)
print("Best RMSE: " + str(minimum_rmse))

52edb24f-905b-4e1d-9ae2-ad624b1754ae 61.91311887374143
abc84bb8-6a98-4825-bec0-d2f2e194b294 61.473546383461105
2a365155-5d54-49c4-9e6d-f9c6107e3778 61.011269045900264
c6b447d4-1f99-4d7d-8e2a-691c491139d9 60.52506485600075
a73424f6-e05e-431d-a9f0-f6cace96443f 60.01417288299607
ee1ca777-d410-4618-b001-f76939fabd9e 59.47884372812509
0955583a-5471-4d93-8804-b359aadd0cba 58.921415390953136
b11ecc1f-6c10-421a-961a-957d5c57d7aa 58.34847059993005
79ecccdd-d5c1-4637-9bc5-aa4177c32d2c 57.77530220002151
ac9c7bd2-ff62-4a30-b937-e71ba09d32ab 57.23482878684463
----------------------------------------------------------------------------------------------------
Best run_id: ac9c7bd2-ff62-4a30-b937-e71ba09d32ab
Best RMSE: 57.23482878684463


In [8]:
from azureml.core import Run

best_run = Run(experiment=experiment, run_id=minimum_rmse_runid)
print(best_run.get_file_names())

['model_alpha_0.1.pkl', 'outputs/model_alpha_0.1.pkl']


In [9]:
best_run.download_file(name = str(best_run.get_file_names()[0]))

In [10]:
import numpy as np
from azureml.core import Dataset
np.savetxt('features.csv', x_train, delimiter=',')
np.savetxt('labels.csv', y_train, delimiter=',')
datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv', './labels.csv'],
                       target_path='diabetes-experiment/',
                       overwrite=True)
input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/labels.csv')])

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of an estimated total of 2
Uploading ./labels.csv
Uploaded ./labels.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [11]:
import sklearn 
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(
                    workspace=ws,
                    model_name="diabetes-experiment-model",
                    model_path=f"./{str(best_run.get_file_names()[0])}",
                    model_framework=Model.Framework.SCIKITLEARN,
                    model_framework_version=sklearn.__version__,
                    sample_input_dataset=input_dataset,
                    sample_output_dataset=output_dataset,
                    resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                    description="Ridge regression model to predict diabetes progression",
                    tags={"area": "diabetes", "type":"regression"}
                    )

Registering model diabetes-experiment-model


In [12]:
service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], overwrite=True)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_375331/2962049591.py:3: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, service_name, [model], overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-09 03:08:20+00:00 Creating Container Registry if not exists..
2025-04-09 03:18:20+00:00 Registering the environment..
2025-04-09 03:18:22+00:00 Uploading autogenerated assets for no-code-deployment.
2025-04-09 03:18:25+00:00 Building image..
2025-04-09 03:26:23+00:00 Generating deployment configuration..
2025-04-09 03:26:25+00:00 Submitting deployment to compute..
2025-04-09 03:26:34+00:00 Checking the status of deployment diabetes-service..
2025-04-09 03:29:05+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
import json
input_payload = json.dumps({
    'data': x_train[0:2].values.tolist(),
    'method': 'predict'
})
output = service.run(input_payload)
print(output)

Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '54', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 09 Apr 2025 03:30:00 GMT', 'Server': 'nginx', 'X-Ms-Client-Request-Id': 'e965ab63-067f-40f8-8c26-9d0f7a0585a4', 'X-Ms-Request-Id': 'e965ab63-067f-40f8-8c26-9d0f7a0585a4', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.7', 'X-Request-Id': 'd9eb3de4-22c7-45b6-b723-e3e0ab6c34f3'}
Content: b"run() missing 1 required positional argument: 'method'"



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '54', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 09 Apr 2025 03:30:00 GMT', 'Server': 'nginx', 'X-Ms-Client-Request-Id': 'e965ab63-067f-40f8-8c26-9d0f7a0585a4', 'X-Ms-Request-Id': 'e965ab63-067f-40f8-8c26-9d0f7a0585a4', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.7', 'X-Request-Id': 'd9eb3de4-22c7-45b6-b723-e3e0ab6c34f3'}
Content: b"run() missing 1 required positional argument: 'method'"
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '54', 'Content-Type': 'text/html; charset=utf-8', '